# Preprocessor Tuning

## (0) The `tumors` Dataset

* 👩🏻‍⚕️ The following dataset describes tumors that are either <font color=red>malignant</font> or <font color=green>benign</font>. 
* 🎯 The task is to detect as many malignant tumors as possible.

In [29]:
import pandas as pd
pd.set_option('display.max_columns', None)

url = "https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/08-Workflow/tumors_dataset.csv"
data = pd.read_csv(url)

data.head()


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,malignant
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,NaN,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,1
4,20.29,14.34,NaN,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.2,1575.0,NaN,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [30]:
round(data.malignant.value_counts(normalize = True),2)


0    0.63
1    0.37
Name: malignant, dtype: float64

In [31]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              568 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           567 non-null    float64
 3   mean area                568 non-null    float64
 4   mean smoothness          568 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           567 non-null    float64
 7   mean concave points      567 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            567 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

## (1) Building a Pipeline

❓ **Question: Building a Pipeline** ❓

Combine the following steps in a **`Pipeline`** object named `pipeline`:

1. Impute missing values with a **`KNNImputer`**
2. Scale all the (numerical) features with a **`MinMaxScaler`**
3. Model a **`LogisticRegression`** with default parameters

In [32]:
# Combine the following steps in a Pipeline object named `pipeline`:
# 1. Impute missing values with a **`KNNImputer`**
# 2. Scale all the (numerical) features with a **`MinMaxScaler`**
# 3. Model a **`LogisticRegression`** with default parameters


from sklearn.impute import KNNImputer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler


pipeline = make_pipeline(KNNImputer(), MinMaxScaler(), LogisticRegression())
pipeline


Pipeline(steps=[('knnimputer', KNNImputer()), ('minmaxscaler', MinMaxScaler()),
                ('logisticregression', LogisticRegression())])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

pipeline.fit(X_train, y_train)

pipeline.predict(X_test)

pipeline.score(X_test, y_test)


(455, 30) (114, 30) (455,) (114,)


0.956140350877193

## (2) Optimizing a pipelined model

❓ **Question (GridSearching a Pipeline)** ❓

* What is the optimal number of neighbors for the KNN imputer: 2, 5, or 10 ? 
    * Perform a GridSearch on your pipeline and save your answer under a variable called `n_best`.
    * _Be careful: Use a scoring metric that is relevant for the task in your Grid Search, just saying... :)_
* Feel free to GridSearch on the whole dataset instead of using a train/test split in this challenge. Here, the goal is just to become familiar with Pipelines :)



In [49]:
n_best = 5


In [45]:
pipeline.get_params()


{'memory': None,
 'steps': [('knnimputer', KNNImputer()),
  ('minmaxscaler', MinMaxScaler()),
  ('logisticregression', LogisticRegression())],
 'verbose': False,
 'knnimputer': KNNImputer(),
 'minmaxscaler': MinMaxScaler(),
 'logisticregression': LogisticRegression(),
 'knnimputer__add_indicator': False,
 'knnimputer__copy': True,
 'knnimputer__keep_empty_features': False,
 'knnimputer__metric': 'nan_euclidean',
 'knnimputer__missing_values': nan,
 'knnimputer__n_neighbors': 5,
 'knnimputer__weights': 'uniform',
 'minmaxscaler__clip': False,
 'minmaxscaler__copy': True,
 'minmaxscaler__feature_range': (0, 1),
 'logisticregression__C': 1.0,
 'logisticregression__class_weight': None,
 'logisticregression__dual': False,
 'logisticregression__fit_intercept': True,
 'logisticregression__intercept_scaling': 1,
 'logisticregression__l1_ratio': None,
 'logisticregression__max_iter': 100,
 'logisticregression__multi_class': 'auto',
 'logisticregression__n_jobs': None,
 'logisticregression__pena

In [48]:
# YOUR CODE HERE
# Use GridSearchCV to find the best number of neighbors for the KNNImputer

from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(
    pipeline, param_grid={"knnimputer__n_neighbors": range(5, 8)}, cv=5,
    n_jobs=-1
)
grid.fit(X_train, y_train)
grid.best_params_


{'knnimputer__n_neighbors': 5}

In [37]:
# YOUR CODE HERE


## (3) Evaluating a pipeline

❓ **Question: what is the performance of the optimal pipeline**  ❓

- Make sure you cross-validate your optimal pipeline! 
- Store your result as a `float` number in a variable named `cv_score`

In [50]:
cv_score = 0.9736263736263735


In [46]:
# YOUR CODE HERE
# Cross validate the optimal pipeline with the best number of neighbors

from sklearn.model_selection import cross_val_score

cv_score = cross_val_score(pipeline, X_train, y_train, cv = 5)
cv_score.mean()


0.9736263736263735

In [51]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solution',
    n_best = n_best,
    cv_score=cv_score
)

result.write()
print(result.check())



============================= test session starts ==============================
platform darwin -- Python 3.10.6, pytest-7.1.3, pluggy-1.0.0 -- /Users/tanushrinayak/.pyenv/versions/3.10.6/envs/lewagon/bin/python3
cachedir: .pytest_cache
rootdir: /Users/tanushrinayak/code/tanushrin/05-ML/08-Workflow/data-preprocessor-tuning/tests
plugins: asyncio-0.19.0, typeguard-2.13.3, anyio-3.6.2
asyncio: mode=strict
collecting ... collected 2 items

test_solution.py::TestSolution::test_n_neighbours PASSED                 [ 50%]
test_solution.py::TestSolution::test_score_good_enough PASSED            [100%]

============================== 2 passed in 0.36s ===============================


💯 You can commit your code:

git add tests/solution.pickle

git commit -m 'Completed solution step'

git push origin master



## (4) Predicting using a fitted and pipelined model

👇 Here is a new tumor.

In [52]:
new_url = "https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/08-Workflow/new_tumor.csv"

new_data = pd.read_csv(new_url)
new_data


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.186,0.275,0.08902


❓ **Question: Using your optimal pipeline, predict whether the new tumor is malignant or not** ❓

In [53]:
# YOUR CODE HERE
# Use optimal pipeline to predict the new data

pipeline.predict(new_data)



array([1])

🏁 Congratulations! You are now an expert at pipelining !

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge!